In [11]:
def calBitOps(num_samples_each_image):
    """
    只统计乘法
    """
    T = 1e12
    sum = 0
    image_size = (800, 800)
    ray_num = image_size[0] * image_size[1]
    per_ray_bitOps = 0  # 每条光线需要的bit操作
    per_point_bitOps = 0  # 每个采样点需要的bit操作
    # (1)------Interpolation------
    # ------Trilinear Interpolation------
    L = 16
    F = 2
    hash_bit = 1
    cord_bit = 32
    TriInterOps = 0
    TriInterOps += 3 * cord_bit * cord_bit
    TriInterOps += L * F * ((4 + 8) * cord_bit * cord_bit + 8 * cord_bit * hash_bit)
    per_point_bitOps += TriInterOps
    print('TriInterOps per point:',TriInterOps)
        # (1-2)------Bilinear Interpolation------
    L = 4
    F = 2
    hash_bit = 1
    cord_bit = 32
    BiInterOps = 0
    BiInterOps += 2 * cord_bit * cord_bit
    BiInterOps += L * F * ((2 + 4) * cord_bit * cord_bit + 4 * cord_bit * hash_bit)
    per_point_bitOps += BiInterOps * 3  # 三个2D网格
    print('BiInterOps per point:',BiInterOps)
    # (2)------SigmaNet------
    # SigmaNet：
    sigma_net_bit = 16
    sigma_net_BitOps = 0
    # layer 0
    sigma_net_BitOps += (80 * 128) * (sigma_net_bit ** 2)
    # layer 1
    sigma_net_BitOps += (128 * 16) * (sigma_net_bit ** 2)
    per_point_bitOps += sigma_net_BitOps
    print('sigma_net_BitOps per point:',sigma_net_BitOps)
    # (3)------ColorNet------
    # ColorNet：
    color_net_bit = 13
    color_net_BitOps = 0
    # layer 0
    color_net_BitOps += (31 * 128) * (color_net_bit ** 2)
    # layer 1
    color_net_BitOps += (128 * 128) * (color_net_bit ** 2)
    # layer 2
    color_net_BitOps += (128 * 3) * (color_net_bit ** 2)
    per_point_bitOps += color_net_BitOps
    print('color_net_BitOps per point:',color_net_BitOps)
    # (4)------volume rendering------
    volRenderOps = 0
    num_samples_per_ray = num_samples_each_image / ray_num
    print('samples per ray:',num_samples_per_ray)
    volRenderOps += sigma_net_bit * num_samples_per_ray * (num_samples_per_ray - 1) / 2
    volRenderOps += num_samples_per_ray * sigma_net_bit * color_net_bit
    print('volRenderOps per ray:',volRenderOps)
    # ------Sum------
    per_ray_bitOps = per_point_bitOps * num_samples_per_ray
    per_ray_bitOps += volRenderOps
    sum = per_ray_bitOps * ray_num
    print('bitOps per image:',sum / T, 'T')
    return sum / T
    
# for i in range(8):
#     calBitOps(i)
#     print('-'*20)

In [12]:
import os
import json
log_dir = 'logs'
# 读取数据
result_folds = [os.path.join(log_dir, d) for d in os.listdir(log_dir) if os.path.isdir(os.path.join(log_dir, d))]
data = {}
for result_fold in result_folds:
    scene_name = result_fold.split('/')[-1]
    json_path = os.path.join(result_fold, 'results.json')
    if os.path.exists(json_path):
        with open(json_path, 'r') as file:
            temp = json.load(file)
        data[scene_name] = temp
    else:
        print(json_path)

logs/bonsai_b_2/results.json


In [8]:
# 计算BitOps
for key in data.keys():
    print('-'*20)
    data[key]['BitOps'] = calBitOps(data[key]['num_samples_each_image'])

--------------------
TriInterOps per point: 404480
BiInterOps per point: 52224
sigma_net_BitOps per point: 3145728
color_net_BitOps per point: 3504384
samples per ray: 275.7244933238636
volRenderOps per ray: 663336.8684143831
bitOps per image: 1272.9506876755408 T
--------------------
TriInterOps per point: 404480
BiInterOps per point: 52224
sigma_net_BitOps per point: 3145728
color_net_BitOps per point: 3504384
samples per ray: 157.50116764322917
volRenderOps per ray: 229953.17600049046
bitOps per image: 727.0479701501337 T
--------------------
TriInterOps per point: 404480
BiInterOps per point: 52224
sigma_net_BitOps per point: 3145728
color_net_BitOps per point: 3504384
samples per ray: 9.381276328124999
volRenderOps per ray: 2580.322029982108
bitOps per image: 43.29820197189758 T
--------------------
TriInterOps per point: 404480
BiInterOps per point: 52224
sigma_net_BitOps per point: 3145728
color_net_BitOps per point: 3504384
samples per ray: 6.5981095234375
volRenderOps per ray:

In [9]:
# 转化成dataframe
import pandas as pd
df = pd.DataFrame(data)  # 使用.T来转置DataFrame，使内部字典的键成为列名
df.to_csv('data.csv')

In [10]:
df


,flowers_b_2,garden_b_2,drums_b_2,Toad_b_2,lego_b_2,materials_b_2,Lifestyle_b_2,Truck_b_2,Caterpillar_b_2,Wineholder_b_2,...,Palace_b_2,hotdog_b_2,Ignatius_b_2,ship_b_2,Robot_b_2,counter_b_2,Spaceship_b_2,mic_b_2,Bike_b_2,bicycle_b_2
psnr,16.63,20.43,2.580000e+01,3.288000e+01,3.478000e+01,2.918000e+01,3.391000e+01,2.697000e+01,2.543000e+01,3.155000e+01,...,3.563000e+01,3.639000e+01,2.781000e+01,2.947000e+01,3.571000e+01,23.6,3.711000e+01,3.560000e+01,3.766000e+01,17.9
ssim,0.274,0.443,9.290000e-01,9.660000e-01,9.730000e-01,9.390000e-01,9.620000e-01,9.010000e-01,8.970000e-01,9.590000e-01,...,9.690000e-01,9.760000e-01,9.430000e-01,8.840000e-01,9.910000e-01,0.78,9.840000e-01,9.880000e-01,9.890000e-01,0.485
lpips,0.606,0.495,7.700000e-02,3.400000e-02,2.900000e-02,7.200000e-02,4.700000e-02,1.480000e-01,1.510000e-01,4.700000e-02,...,2.300000e-02,3.900000e-02,7.700000e-02,1.300000e-01,1.100000e-02,0.473,2.300000e-02,1.800000e-02,1.500000e-02,0.452
test_time,106.07,71.1,8.644000e+01,6.252000e+01,8.120000e+01,9.173000e+01,8.877000e+01,9.080000e+01,1.306700e+02,6.182000e+01,...,7.253000e+01,1.000000e+02,4.821000e+01,1.806400e+02,5.655000e+01,1239.65,5.549000e+01,6.224000e+01,5.788000e+01,25.25
render_speed,0.21,0.34,2.310000e+00,3.200000e+00,2.460000e+00,2.180000e+00,2.250000e+00,3.500000e-01,3.500000e-01,3.240000e+00,...,2.760000e+00,2.000000e+00,6.800000e-01,1.110000e+00,3.540000e+00,0.02,3.600000e+00,3.210000e+00,3.460000e+00,0.95
size,1.883,1.888,1.524500e+00,1.600300e+00,1.459800e+00,1.506200e+00,1.614000e+00,1.654900e+00,1.687000e+00,1.507000e+00,...,1.599100e+00,1.476100e+00,1.699700e+00,1.610900e+00,1.530200e+00,1.9202,1.629400e+00,1.385200e+00,1.587600e+00,1.8831
encoding_size,1.6361,1.6599,1.440700e+00,1.477700e+00,1.379500e+00,1.430500e+00,1.517400e+00,1.542200e+00,1.570400e+00,1.418400e+00,...,1.491500e+00,1.395400e+00,1.591400e+00,1.508900e+00,1.441900e+00,1.6551,1.525200e+00,1.311100e+00,1.492700e+00,1.6376
network_size,0.0685,0.0685,6.850000e-02,6.850000e-02,6.850000e-02,6.850000e-02,6.850000e-02,6.850000e-02,6.850000e-02,6.850000e-02,...,6.850000e-02,6.850000e-02,6.850000e-02,6.850000e-02,6.850000e-02,0.0685,6.850000e-02,6.850000e-02,6.850000e-02,0.0685
occgrid_size,0.1784,0.1595,1.520000e-02,5.410000e-02,1.170000e-02,7.100000e-03,2.810000e-02,4.430000e-02,4.810000e-02,2.010000e-02,...,3.900000e-02,1.230000e-02,3.970000e-02,3.350000e-02,1.970000e-02,0.1966,3.570000e-02,5.600000e-03,2.640000e-02,0.1769
num_samples_each_image,176463675.727273,100800747.291667,6.004017e+06,4.222790e+06,4.515144e+06,9.884541e+06,1.018019e+07,5.570991e+07,5.370977e+07,4.667399e+06,...,6.674275e+06,4.774743e+06,3.293183e+07,1.946656e+07,1.714795e+06,739236207.8,4.095864e+06,3.024713e+06,1.910852e+06,48497365.541667
